In [1]:
import numpy as np, pandas as pd, plotly.express as px, plotly.graph_objects as go
from scipy.signal import savgol_filter as sgf
from pathlib import Path
import scipy.optimize as opt

from SJ_Haar_CNV import base_search
from SJ_Haar_CNV import decomposition
from tests import test_data

In [25]:
X = np.concatenate([np.repeat (5,180), np.repeat (6,21), np.repeat (5,190), np.repeat (6,190), np.repeat(5,100)])+ np.random.normal(0,1,681)
l = 681
#

In [26]:
basis = base_search.generate_haar_basis(X, p0=.8, length = 21, debug = False)
basis

[[0, 0, 0, (5.273815721170443, 681), (0, 0)],
 [0, 681, 0, (0.033034362370253215, 390), (-0.04438637693047768, 291)],
 [1, 390, 0, (0.05503678244354135, 178), (-0.04646973611978255, 212)],
 [1, 290, 391, (0.0420940282759306, 191), (-0.08163690332301694, 99)],
 [2, 178, 0, (0.06872180273933821, 96), (-0.08129286421604644, 82)],
 [2, 211, 179, (0.11191245765449598, 57), (-0.042148847688056934, 154)],
 [2, 191, 391, (0.13849877461228924, 40), (-0.037605627543734164, 151)],
 [2, 98, 583, (0.16412198797244362, 26), (-0.061545745489666355, 72)],
 [3, 96, 0, (0.07800105302132374, 60), (-0.13216845095279856, 36)],
 [3, 81, 97, (0.11595647325529579, 38), (-0.10516982458038455, 43)],
 [3, 57, 179, (0.17429190545949858, 20), (-0.09892243282836406, 37)],
 [3, 153, 237, (0.06263513529290228, 95), (-0.10367194807101067, 58)],
 [3, 40, 391, (0.2944840695329241, 8), (-0.0828236445561349, 32)],
 [3, 150, 432, (0.12105403416817409, 46), (-0.054707111595232526, 104)],
 [3, 26, 583, (0.3600411499115478, 5

In [27]:
res = decomposition.decompose (X, basis)

In [28]:
basis1, coefficients, unneccessary_remove_later, rec_signal = res

In [36]:
coefficients[0] = 1

In [37]:
decomposition.generate_function_from_wavelets (coefficients, basis, l)

array([4.32693384, 4.32693384, 4.32693384, 4.32693384, 4.32693384,
       4.32693384, 4.32693384, 4.32693384, 4.32693384, 5.01509196,
       4.7556124 , 4.7556124 , 4.7556124 , 4.7556124 , 4.7556124 ,
       4.7556124 , 4.7556124 , 4.7556124 , 4.7556124 , 4.7556124 ,
       4.7556124 , 5.1707797 , 5.1707797 , 5.1707797 , 5.1707797 ,
       5.1707797 , 5.1707797 , 5.1707797 , 5.1707797 , 5.1707797 ,
       5.1707797 , 5.1707797 , 5.1707797 , 5.1707797 , 5.1707797 ,
       5.1707797 , 5.1707797 , 5.1707797 , 5.1707797 , 5.1707797 ,
       5.1707797 , 4.54776121, 4.54776121, 4.54776121, 4.54776121,
       4.54776121, 4.54776121, 4.54776121, 4.54776121, 4.54776121,
       4.54776121, 4.54776121, 4.54776121, 4.54776121, 4.54776121,
       4.54776121, 4.54776121, 4.54776121, 4.54776121, 4.54776121,
       5.374519  , 4.84609922, 4.84609922, 4.84609922, 4.84609922,
       4.84609922, 4.84609922, 4.84609922, 4.84609922, 4.84609922,
       4.84609922, 5.60702371, 5.25897693, 5.25897693, 5.25897

In [38]:
fig = go.Figure()
#fig.add_trace(go.Scatter(x=df['location'], y=df['raw'], name='Raw Signal', mode='markers',marker=markers))
fig.add_trace(go.Scatter(y = X, mode = 'markers'))
fig.add_trace (go.Scatter (y = decomposition.generate_function_from_wavelets (coefficients, basis, l)))
ind = np.array([0,   1,3])
#allind = np.arange(len(basis))
fig.add_trace (go.Scatter (y =decomposition.generate_function_from_wavelets (coefficients[ind], [basis[i] for i in ind], l)))

In [43]:
px.plot (y = X - decomposition.generate_function_from_wavelets (coefficients[ind], [basis[i] for i in ind], l))

AttributeError: module 'plotly.express' has no attribute 'plot'

In [42]:
fig = go.Figure()
v = np.abs (coefficients)


def exp_cdf (x, a):
    return 1- np.exp(-a*x)

popt, pcov = opt.curve_fit(exp_cdf, np.sort(v-1), np.linspace(0, 1, len(v)), p0=[1])


#fig.add_trace(go.Scatter(x=df['location'], y=df['raw'], name='Raw Signal', mode='markers',marker=markers))
#fig.add_trace(go.Scatter(y = X, mode = 'markers'))
fig.add_trace(go.Scatter (x = np.sort(v-1), y = np.linspace (0,1,len(v)), mode = 'markers'))

c = np.linspace (0, 18, 100)
fig.add_trace(go.Scatter (x = c, y = exp_cdf(c, *popt)))


In [34]:
np.argsort(v)[-5:]

array([ 8, 20, 33,  1,  3], dtype=int64)

In [111]:
fig = go.Figure()
fig.add_trace(go.Scatter (x = np.sort(np.log(v)), y = np.linspace (0,1,len(v)), mode = 'markers'))